In [2]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('players.db')

In [3]:
def format_duration(seconds):
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{int(hours):02d}h {int(minutes):02d}m {int(seconds):02d}s"



## TF2 Surveillance war room

In [49]:
#query recent sessions

query = """
SELECT sessions.*, players.name AS player_name, servers.name AS server_name, servers.address AS server_address
FROM sessions
JOIN players ON sessions.player_id = players.player_id
JOIN servers ON sessions.server_id = servers.server_id;
"""
all_sessions = pd.read_sql_query(query, conn)
all_sessions["duration"] = all_sessions["duration"].apply(format_duration)

print(all_sessions.shape)

all_sessions.tail()

(4759, 10)


,session_id,server_id,player_id,score,duration,joined_at,left_at,player_name,server_name,server_address
4754,4755,2,407864,0,00h 43m 00s,2023-11-06 14:42:33,2023-11-06 15:25:33,Beechet,Uncletopia | Sydney | 1 | All Maps,51.161.198.97:27015
4755,4756,75,14,0,144h 02m 27s,2023-10-31 15:23:06,2023-11-06 15:25:33,GENTLE MANNE of LEISURE,Obamaworld FF2,45.121.209.68:27500
4756,4757,75,15,0,144h 02m 27s,2023-10-31 15:23:06,2023-11-06 15:25:33,Tiny Baby Man,Obamaworld FF2,45.121.209.68:27500
4757,4758,75,16,1,144h 02m 27s,2023-10-31 15:23:06,2023-11-06 15:25:33,Force of Nature,Obamaworld FF2,45.121.209.68:27500
4758,4759,84,435776,31,00h 08m 25s,2023-11-06 15:17:22,2023-11-06 15:25:47,Ne-Yo,skial.com | 2FORT | AU...,91.216.250.194:27015


In [47]:
#Sessions by player name

query = """
SELECT sessions.*, servers.name AS server_name, servers.address AS server_address 
FROM sessions 
JOIN players ON sessions.player_id = players.player_id 
JOIN servers ON sessions.server_id = servers.server_id 
WHERE players.name = 'Qwerty AFK';
"""
sessions_by_player = pd.read_sql_query(query, conn)

sessions_by_player["duration"] = sessions_by_player["duration"].apply(format_duration)

sessions_by_player.head()

,session_id,server_id,player_id,score,duration,joined_at,left_at,server_name,server_address
0,2880,89,282036,349,02h 16m 21s,2023-11-06 08:44:33,2023-11-06 11:00:54,skial.com | 2FORT+ | A...,91.216.250.193:27015
1,3121,89,282036,478,02h 53m 05s,2023-11-06 08:44:33,2023-11-06 11:37:38,skial.com | 2FORT+ | A...,91.216.250.193:27015


In [43]:
#query players total playtime and sessions

query = """SELECT p.player_id, p.name, COUNT(s.session_id) AS session_count, SUM(s.duration) AS total_duration FROM players p JOIN sessions s ON p.player_id = s.player_id GROUP BY p.player_id, p.name HAVING COUNT(s.session_id) > 1;"""
session_count_by_player = pd.read_sql_query(query, conn)
session_count_by_player["total_duration"] = session_count_by_player["total_duration"].apply(format_duration)

print(session_count_by_player.shape)

session_count_by_player.head()

(779, 4)


,player_id,name,session_count,total_duration
0,1,CrySomeMore,8,138h 53m 46s
1,2,Humans Are Weak,9,137h 16m 21s
2,3,C++,6,108h 14m 40s
3,4,Nobody,7,131h 58m 18s
4,5,RageQuit,9,146h 23m 07s


In [46]:
#query player events by name

query = """
SELECT pe.*, s.name AS server_name, s.address AS server_address
FROM player_events AS pe
JOIN players AS p ON pe.player_id = p.player_id
JOIN servers AS s ON pe.server_id = s.server_id
WHERE p.name = 'elmaxo';
"""
events_by_name = pd.read_sql_query(query, conn)

print(events_by_name.shape)

events_by_name.head()

(2, 7)


,event_id,server_id,player_id,event_type,created_at,server_name,server_address
0,22455,3,399538,target join,2023-11-06 14:32:12,Uncletopia | Sydney | 2 | All Maps,51.161.198.97:27025
1,22663,3,399538,target join,2023-11-06 14:32:54,Uncletopia | Sydney | 2 | All Maps,51.161.198.97:27025
